In [1]:
#Import relevant libraries
import pandas as pd
import numpy as np

In [191]:
#Load data
xlsx = pd.ExcelFile("HSC My Journey data.xlsx")
df1 = pd.read_excel(xlsx, "My Journey", skiprows=[0])
df2 = pd.read_excel(xlsx, "Interaction")

In [192]:
#Examine datasheet
df1.head()

,MyJourney ID,Interaction ID,Confidence Start,Confidence End,How much did the programme help you develop this skill,Communication Start,Communication End,How much did the programme help you develop this skill.1,Working with others Start,Working with others End,...,Reliability End,How much did the programme help you develop this skill.4,Setting & achieving goals Start,Setting & achieving goals End,How much did the programme help you develop this skill.5,How much has the programme helped how satisfied you feel about life,Has the programme helped you to achieve your goals,How helpful was the guidance from staff,How much has the programme improved your opportunities for employment/sefl-employment,Would you recommend the Prince's Trust to others
0,141017,801100,3,5,Some,4,5,Some,4,6,...,6,No Value,5,6,Some,Quite a lot,A little,Quite a lot,Quite a lot,Yes
1,226145,787826,6,No Value,Some,6,No Value,Quite a lot,6,No Value,...,No Value,Quite a lot,3,No Value,Quite a lot,A lot,A lot,A lot,Quite a lot,Yes
2,226146,787828,5,6,Quite a lot,4,6,Quite a lot,4,6,...,5,Quite a lot,4,6,Quite a lot,Quite a lot,Quite a lot,Quite a lot,Quite a lot,Yes
3,226147,787827,3,5,Some,3,4,Some,4,5,...,3,A little,2,5,A little,A little,A little,A little,A little,Yes
4,226152,787835,4,No Value,Quite a lot,4,5,Quite a lot,5,No Value,...,5,Quite a lot,5,No Value,Quite a lot,Quite a lot,Quite a lot,Quite a lot,Quite a lot,No Value


In [193]:
#Check number of participants
df1.shape

(20399, 25)

In [147]:
#Check for duplicates
df1.duplicated().value_counts()

False    20399
dtype: int64

20,399 young people included in the data

In [148]:
#Check for any empty values
df1.isnull().values.any()

False

In [134]:
#Examine the second data sheet
df2.head()

,Interaction ID,SuperRegion,Region,SubRegion
0,787769,North,North West,Greater Manchester
1,787812,North,North West,Greater Manchester
2,787822,North,North West,Merseyside
3,787826,No Value,No Value,No Value
4,787827,No Value,No Value,No Value


In [135]:
#Check number of participants to see if the same as sheet 1
df2.shape

(20399, 4)

In [146]:
#Check for duplicates
df2.duplicated().value_counts()

False    20399
dtype: int64

In [207]:
#Need to merge files on Interacton ID
merged = pd.merge(df1, df2, how = "outer") #Use outer so as not to miss any rows

In [195]:
#Check number of participants is still the same
merged.shape

(20399, 28)

In [206]:
#Change "No Value" to 100 or 10 to make it easier to compute from
merged["Confidence Start"].replace("No Value",10, inplace=True)
merged["Communication Start"].replace("No Value",10, inplace=True)
merged["Working with others Start"].replace("No Value",10, inplace=True)
merged["Managing feelings Start"].replace("No Value",10, inplace=True)
merged["Reliability Start"].replace("No Value",10, inplace=True)

merged["Confidence End"].replace("No Value",100, inplace=True)
merged["Communication End"].replace("No Value",100, inplace=True)
merged["Working with others End"].replace("No Value",100, inplace=True)
merged["Managing feelings End"].replace("No Value",100, inplace=True)
merged["Reliability End"].replace("No Value",100, inplace=True)

#Perform calculation into new columns, to find end score. A score > 6 shows there were null values so this should
#be ignored
merged["Confidence"] = (merged["Confidence End"] - merged["Confidence Start"])
merged["Communication"] = (merged["Communication End"] - merged["Communication Start"])
merged["Working"] = (merged["Working with others End"] - merged["Working with others Start"])
merged["Feelings"] = (merged["Managing feelings End"] - merged["Managing feelings Start"])
merged["Reliability"] = (merged["Reliability End"] - merged["Reliability Start"])

In [197]:
#Create new columns and assign 1 if improvement in skill, 0 if not

def f(row):
    if (row["Confidence"]>0) & (row["Confidence"]<10) :
        val = 1
    else:
        val = 0
    return val

def f2(row):
    if (row["Communication"]>0) & (row["Communication"]<10) :
        val = 1
    else:
        val = 0
    return val

def f3(row):
    if (row["Working"]>0) & (row["Working"]<10) :
        val = 1
    else:
        val = 0
    return val

def f4(row):
    if (row["Feelings"]>0) & (row["Feelings"]<10) :
        val = 1
    else:
        val = 0
    return val

def f5(row):
    if (row["Reliability"]>0) & (row["Reliability"]<10) :
        val = 1
    else:
        val = 0
    return val

merged['Confidence_Increase'] = merged.apply(f, axis=1)
merged['Communication_Increase'] = merged.apply(f2, axis=1)
merged['Working_Increase'] = merged.apply(f3, axis=1)
merged['Feelings_Increase'] = merged.apply(f4, axis=1)
merged['Reliability_Increase'] = merged.apply(f5, axis=1)

#Create new column "Skills Improved" for total number of skills increased
merged["Skills Improved"] = merged['Confidence_Increase'] + merged['Communication_Increase'] + \
merged['Working_Increase'] + merged['Feelings_Increase'] + merged['Reliability_Increase']                         

In [198]:
#Look at superregion names
SuperRegions = merged.SuperRegion.unique()
SuperRegions

array(['Scotland', 'No Value', 'North', 'South', 'Central', 'Wales',
       'Head Office', 'Northern Ireland'], dtype=object)

In [199]:
#Groupby superregions
Scotland = merged[merged.SuperRegion == "Scotland"].reset_index()
North = merged[merged.SuperRegion == "North"].reset_index()
South = merged[merged.SuperRegion == "South"].reset_index()
Central = merged[merged.SuperRegion == "Central"].reset_index()
Wales = merged[merged.SuperRegion == "Wales"].reset_index()
Head_Office = merged[merged.SuperRegion == "Head Office"].reset_index()
Northern_Ireland = merged[merged.SuperRegion == "Northern Ireland"].reset_index()
No_Value = merged[merged.SuperRegion == "No Value"].reset_index()

In [203]:
#Question 1. Find the number and percentage of young people who have improved in 3 or more skills

scotland_three_skill = Scotland[(Scotland["Skills Improved"] >= 3)]
north_three_skill = North[(North["Skills Improved"] >= 3)]
south_three_skill = South[(South["Skills Improved"] >= 3)]
central_three_skill = Central[(Central["Skills Improved"] >= 3)  ] 
wales_three_skill = Wales[(Wales["Skills Improved"] >= 3)]
head_office_three_skill = Head_Office[(Head_Office["Skills Improved"] >= 3)  ]
northern_ireland_three_skill = Northern_Ireland[(Northern_Ireland["Skills Improved"] >= 3)]   
no_value_three_skill = No_Value[(No_Value["Skills Improved"] >= 3)]  
uk_three_skill = merged[(merged["Skills Improved"] >= 3)] 

#Calculate percentage of three skill improvement per SuperRegion
uk_percent_three_skill = (uk_three_skill["Interaction ID"].nunique()/merged["Interaction ID"]\
                                  .nunique())*100
scotland_percent_three_skill = (scotland_three_skill["Interaction ID"]\
                                        .nunique()/Scotland["Interaction ID"].nunique())*100
north_percent_three_skill = (north_three_skill["Interaction ID"]\
                                     .nunique()/North["Interaction ID"].nunique())*100
south_percent_three_skill = (south_three_skill["Interaction ID"]\
                                     .nunique()/South["Interaction ID"].nunique())*100
central_percent_three_skill = (central_three_skill["Interaction ID"]\
                                       .nunique()/Central["Interaction ID"].nunique())*100
wales_percent_three_skill = (wales_three_skill["Interaction ID"]\
                                     .nunique()/Wales["Interaction ID"].nunique())*100
head_office_percent_three_skill = (head_office_three_skill["Interaction ID"]\
                                           .nunique()/Head_Office["Interaction ID"].nunique())*100
northern_ireland_percent_three_skill = (northern_ireland_three_skill["Interaction ID"]\
                                                .nunique()/Northern_Ireland["Interaction ID"].nunique())*100
no_value_percent_three_skill = (no_value_three_skill["Interaction ID"]\
                                        .nunique()/No_Value["Interaction ID"].nunique())*100

In [174]:
#Question 2. Find the number and percentage of young people who have improved in confidence

#Calculate number with improved confidence per superegion
scotland_improved_confidence = Scotland[(Scotland["Confidence"] > 0) & (Scotland["Confidence"] < 10)]
north_improved_confidence = North[(North["Confidence"] > 0) & (North["Confidence"] < 10)]
south_improved_confidence = South[(South["Confidence"] > 0) & (South["Confidence"] < 10)]
central_improved_confidence = Central[(Central["Confidence"] > 0) & (Central["Confidence"] < 10)]
wales_improved_confidence = Wales[(Wales["Confidence"] > 0) & (Wales["Confidence"] < 10)]
head_office_improved_confidence = Head_Office[(Head_Office["Confidence"] > 0) & (Head_Office["Confidence"] < 10)]
northern_ireland_improved_confidence = Northern_Ireland[(Northern_Ireland["Confidence"] > 0) & \
                                                        (Northern_Ireland["Confidence"] < 10)]
no_value_improved_confidence = No_Value[(No_Value["Confidence"] > 0) & (No_Value["Confidence"] < 10)]
uk_improved_confidence = merged[(merged["Confidence"] > 0) & (merged["Confidence"] < 10)]

In [172]:
#Calculate percentage of confidence improvement per SuperRegion
uk_percent_improved_confidence = (uk_improved_confidence["Interaction ID"].nunique()/merged["Interaction ID"]\
                                  .nunique())*100
scotland_percent_improved_confidence = (scotland_improved_confidence["Interaction ID"]\
                                        .nunique()/Scotland["Interaction ID"].nunique())*100
north_percent_improved_confidence = (north_improved_confidence["Interaction ID"]\
                                     .nunique()/North["Interaction ID"].nunique())*100
south_percent_improved_confidence = (south_improved_confidence["Interaction ID"]\
                                     .nunique()/South["Interaction ID"].nunique())*100
central_percent_improved_confidence = (central_improved_confidence["Interaction ID"]\
                                       .nunique()/Central["Interaction ID"].nunique())*100
wales_percent_improved_confidence = (wales_improved_confidence["Interaction ID"]\
                                     .nunique()/Wales["Interaction ID"].nunique())*100
head_office_percent_improved_confidence = (head_office_improved_confidence["Interaction ID"]\
                                           .nunique()/Head_Office["Interaction ID"].nunique())*100
northern_ireland_percent_improved_confidence = (northern_ireland_improved_confidence["Interaction ID"]\
                                                .nunique()/Northern_Ireland["Interaction ID"].nunique())*100
no_value_percent_improved_confidence = (no_value_improved_confidence["Interaction ID"]\
                                        .nunique()/No_Value["Interaction ID"].nunique())*100

In [208]:
#Question 3. Present the above results for each super region, along with the totals for England

#Create a table with all info
final_df = pd.DataFrame([["UK Total", merged["Interaction ID"].nunique(),
                          uk_three_skill["Interaction ID"].nunique(), 
                          uk_percent_three_skill,
                          uk_improved_confidence["Interaction ID"].nunique(), 
                          uk_percent_improved_confidence],
                        ["Scotland", Scotland["Interaction ID"].nunique(),
                         scotland_three_skill["Interaction ID"].nunique(),
                         scotland_percent_three_skill,
                         scotland_improved_confidence["Interaction ID"].nunique(), 
                         scotland_percent_improved_confidence],
                        ["North", North["Interaction ID"].nunique(),
                         north_three_skill["Interaction ID"].nunique(),
                         north_percent_three_skill,
                         north_improved_confidence["Interaction ID"].nunique(),
                         north_percent_improved_confidence],
                        ["South", South["Interaction ID"].nunique(),
                         south_three_skill["Interaction ID"].nunique(),
                         south_percent_three_skill,
                         south_improved_confidence["Interaction ID"].nunique(), 
                         south_percent_improved_confidence],
                        ["Central", Central["Interaction ID"].nunique(),
                         central_three_skill["Interaction ID"].nunique(),
                         central_percent_three_skill,
                         central_improved_confidence["Interaction ID"].nunique(), 
                         central_percent_improved_confidence],
                        ["Wales", Wales["Interaction ID"].nunique(),
                         wales_three_skill["Interaction ID"].nunique(),
                         wales_percent_three_skill,
                         wales_improved_confidence["Interaction ID"].nunique(), 
                         wales_percent_improved_confidence],
                        ["Head Office", Head_Office["Interaction ID"].nunique(),
                         head_office_three_skill["Interaction ID"].nunique(),
                         head_office_percent_three_skill,
                         head_office_improved_confidence["Interaction ID"].nunique(), 
                         head_office_percent_improved_confidence],
                        ["Northern Ireland", Northern_Ireland["Interaction ID"].nunique(),
                         northern_ireland_three_skill["Interaction ID"].nunique(),
                         northern_ireland_percent_three_skill,
                         northern_ireland_improved_confidence["Interaction ID"].nunique(), 
                         northern_ireland_percent_improved_confidence],
                        ["No Value", No_Value["Interaction ID"].nunique(),
                         no_value_three_skill["Interaction ID"].nunique(),
                         no_value_percent_three_skill,
                         no_value_improved_confidence["Interaction ID"].nunique(), 
                         no_value_percent_improved_confidence]
                        ],
                       columns = ["SuperRegion", "No. of young people","No. improved in 3 or more skills",
                                  "Percentage improved 3 or more skills",
                                  "No. with improved confidence", 
                                  "Percentage improved confidence"])
final_df

,SuperRegion,No. of young people,No. improved in 3 or more skills,Percentage improved 3 or more skills,No. with improved confidence,Percentage improved confidence
0,UK Total,20399,7438,36.462572,7518,36.854748
1,Scotland,827,133,16.082225,131,15.840387
2,North,2880,864,30.000000,885,30.729167
3,South,3667,490,13.362422,502,13.689665
4,Central,2902,630,21.709166,628,21.640248
5,Wales,774,197,25.452196,202,26.098191
6,Head Office,364,29,7.967033,32,8.791209
7,Northern Ireland,1008,172,17.063492,178,17.658730
8,No Value,7977,4923,61.714930,4960,62.178764
